In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
#DONE
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from MongoCRUD import MongoCRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
#DONE

username = "aacuser"
password = "Welcome1"

# Connect to database via CRUD Module
db = MongoCRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
#DONE
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
#DONE
image_filename = '/home/michaelevans1_snhu/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'150px'})),
    html.Center(html.B(html.H1('CS-340 Dashboard - Michael Evans - 9/25/24'))),
    html.Hr(),
    html.Div(),
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    #DONE
    # Radio buttons for filtering by rescue type
    html.Label('Rescue Type:'),
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'water_rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'},
            {'label': 'Reset', 'value': 'reset'}  # Reset returns to unfiltered state
        ],
        value='reset',  # Default value is reset (unfiltered)
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
#DONE
        # Enable row selection, pagination, and sorting
        row_selectable='single',
        selected_rows=[0],  # Select the first row by default
        page_size=10,  # Limit the number of rows per page
        sort_action="native",  # Enable sorting on columns
        style_data_conditional=[
            # Style the selected row
            {
                'if': {'row_index': 0},  # Style the default selected row
                'backgroundColor': '#D2F3FF',
            }
        ],
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H3("Michael Evans SNHU CS-340")
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-filter', 'value')])
def update_dashboard(filter_type):
    # Dictionary mapping filter types to MongoDB query criteria
    filter_criteria = {
        'water_rescue': {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        },
        'mountain_rescue': {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        },
        'disaster_tracking': {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        },
        'reset': {}  # No filter for reset; this returns all records
    }

    # Get the query based on the selected filter_type
    query = filter_criteria.get(filter_type, {})

    # Fetch data from MongoDB using the query
    filtered_data = db.read(query)  # Ensure db.read returns a list of dictionaries

    # Remove the '_id' field which causes serialization issues
    for record in filtered_data:
        record.pop('_id', None)  # Safely remove '_id' from each record

    # Return the filtered data for the DataTable
    return filtered_data

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Check if there's any data to display
    if viewData is None or len(viewData) == 0:
        return []

    # Convert the viewData (a list of dictionaries) to a pandas DataFrame
    df_filtered = pd.DataFrame(viewData)

    # Group by breed and count the occurrences
    breed_count = df_filtered['breed'].value_counts().reset_index()
    breed_count.columns = ['breed', 'count']  # Rename columns for convenience

    # Generate the pie chart based on the filtered data (adjust the column names as needed)
    fig = px.pie(
        breed_count, 
        names='breed',  # Column with the labels (breed)
        values='count',  # Column with the counts for each breed
        title='Preferred Animals'
    )

    # Customize the layout and colors to match the style in the image
    fig.update_traces(
        textposition='inside', 
        textinfo='percent+label',  # Show percentage and label inside the chart
        pull=[0.05 if i == 0 else 0 for i in range(len(breed_count))],  # Pull the first slice slightly outward (optional)
        marker=dict(
            colors=px.colors.sequential.Reds,  # You can change the color scheme to a different one if needed
            line=dict(color='white', width=2)  # White borders around the slices
        )
    )

    # Customize the layout (title, font, legend)
    fig.update_layout(
        showlegend=True,  # Ensure legend is displayed
        legend_title_text='Breeds',  # Customize legend title
        title_x=0.5,  # Center the title
        title_font_size=24,  # Title font size
        legend=dict(
            x=1, 
            y=1,
            traceorder='normal',
            font=dict(size=12),
        )
    )

    # Return the graph as a Dash component
    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None or len(selected_columns) == 0:
        return []  # Return no styles if no columns are selected

    # If columns are selected, apply the background color style
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to display on the map.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None or len(index) == 0: #should help clear up an error i've been getting
        row = 0
    else: 
        row = index[0]
        
        
    # Ensure that the lat/lon columns are valid and exist
    lat_col, lon_col = 13, 14  # column indinces for these when i debug
    if lat_col in dff.columns and lon_col in dff.columns:
        lat = dff.iloc[row, lat_col]
        lon = dff.iloc[row, lon_col]
    else:
        # Default to Austin, TX coordinates if lat/lon not found
        lat, lon = 30.75, -97.48
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[lat, lon], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               # Marker with tool tip and popup
               # Column 13 and 14 define the grid-coordinates for 
               # the map
               # Column 4 defines the breed for the animal
               # Column 9 defines the name of the animal
               dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                  children=[
                      dl.Tooltip(dff.iloc[row,4]),
                      dl.Popup([
                         html.H1("Animal Name"),
                        html.P(dff.iloc[row,9])
                     ])
                  ])
           ])
        ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:27087/
